# Introduction
What is the purpose of this project 

In [2]:
import pandas as pd 

# Loading the Data


In [4]:
# Read the CSV file 

stack_overflow_file_path = 'developer_dataset.csv'

try:
    developer_dataset = pd.read_csv(stack_overflow_file_path)
except FileNotFoundError:
    print(f'Error: the file {stack_overflow_file_path} was not found')

stack_overflow_df = developer_dataset.copy()

/var/folders/ss/_s30b2md5nsgx4nr60qrj4nc0000gn/T/ipykernel_8235/942379579.py:6: DtypeWarning: Columns (18,19,20) have mixed types. Specify dtype option on import or set low_memory=False.
  developer_dataset = pd.read_csv(stack_overflow_file_path)


The dataset has 111,209 entries and 21 columns. Here are the first few rows and an overview of the columns:

	•	RespondentID: Unique identifier for each respondent.
	•	Year: The year the survey was conducted.
	•	Country: The respondent’s country.
	•	Employment: Employment status.
	•	UndergradMajor: Undergraduate major.
	•	DevType: Types of developers.
	•	LanguageWorkedWith: Languages the respondent has worked with.
	•	LanguageDesireNextYear: Languages the respondent wants to work with next year.
	•	DatabaseWorkedWith: Databases the respondent has worked with.
	•	DatabaseDesireNextYear: Databases the respondent wants to work with next year.
	•	PlatformWorkedWith: Platforms the respondent has worked with.
	•	PlatformDesireNextYear: Platforms the respondent wants to work with next year.
	•	Hobbyist: Whether the respondent codes as a hobby.
	•	OrgSize: Size of the organization the respondent works for.
	•	YearsCodePro: Years of professional coding experience.
	•	JobSeek: Whether the respondent is looking for a job.
	•	ConvertedComp: Annual salary in USD.
	•	WorkWeekHrs: Hours worked per week.
	•	NEWJobHunt: Reason for job hunting.
	•	NEWJobHuntResearch: How the respondent searches for new jobs.
	•	NEWLearn: How the respondent learns new technologies.


In [6]:
# Check the DataFrame info:
stack_overflow_df.info()
# Display the first few rows of the dataset
stack_overflow_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111209 entries, 0 to 111208
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   RespondentID            111209 non-null  int64  
 1   Year                    111209 non-null  int64  
 2   Country                 111209 non-null  object 
 3   Employment              109425 non-null  object 
 4   UndergradMajor          98453 non-null   object 
 5   DevType                 100433 non-null  object 
 6   LanguageWorkedWith      102018 non-null  object 
 7   LanguageDesireNextYear  96044 non-null   object 
 8   DatabaseWorkedWith      85859 non-null   object 
 9   DatabaseDesireNextYear  74234 non-null   object 
 10  PlatformWorkedWith      91609 non-null   object 
 11  PlatformDesireNextYear  85376 non-null   object 
 12  Hobbyist                68352 non-null   object 
 13  OrgSize                 54804 non-null   object 
 14  YearsCodePro        

,RespondentID,Year,Country,Employment,UndergradMajor,DevType,LanguageWorkedWith,LanguageDesireNextYear,DatabaseWorkedWith,DatabaseDesireNextYear,...,PlatformDesireNextYear,Hobbyist,OrgSize,YearsCodePro,JobSeek,ConvertedComp,WorkWeekHrs,NEWJobHunt,NEWJobHuntResearch,NEWLearn
0,1,2018,United States,Employed full-time,"Computer science, computer engineering, or sof...",Engineering manager;Full-stack developer,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,141000.0,NaN,NaN,NaN,NaN
1,1,2019,United States,Employed full-time,"Computer science, computer engineering, or sof...","Developer, full-stack",C;C++;C#;Python;SQL,C;C#;JavaScript;SQL,MySQL;SQLite,MySQL;SQLite,...,Linux;Windows,No,100 to 499 employees,1.0,I am not interested in new job opportunities,61000.0,80.0,NaN,NaN,NaN
2,1,2020,United States,Employed full-time,"Computer science, computer engineering, or sof...",NaN,HTML/CSS;Ruby;SQL,Java;Ruby;Scala,MySQL;PostgreSQL;Redis;SQLite,MySQL;PostgreSQL,...,Docker;Google Cloud Platform;Heroku;Linux;Windows,Yes,NaN,8.0,NaN,NaN,NaN,NaN,NaN,Once a year
3,2,2018,United States,Employed full-time,"Computer science, computer engineering, or sof...",Full-stack developer,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,C#;JavaScript;SQL;TypeScript;HTML;CSS;Bash/Shell,"SQL Server;Microsoft Azure (Tables, CosmosDB, ...","SQL Server;Microsoft Azure (Tables, CosmosDB, ...",...,Azure,NaN,NaN,4.0,NaN,48000.0,NaN,NaN,NaN,NaN
4,2,2019,United States,Employed full-time,"Computer science, computer engineering, or sof...",Data or business analyst;Database administrato...,Bash/Shell/PowerShell;HTML/CSS;JavaScript;PHP;...,Bash/Shell/PowerShell;HTML/CSS;JavaScript;Rust...,Couchbase;DynamoDB;Firebase;MySQL,Firebase;MySQL;Redis,...,Android;AWS;Docker;IBM Cloud or Watson;Linux;S...,Yes,10 to 19 employees,8.0,I am not interested in new job opportunities,90000.0,40.0,NaN,NaN,NaN


# Data Cleaning

In [7]:
# Check for missing values in each column 
missing_data = stack_overflow_df.isnull().sum()
missing_data_percentage = (missing_data / len(stack_overflow_df)) * 100 
missing_summary = pd.DataFrame({
    'Missing Values': missing_data,
    'Percentage': missing_data_percentage
})

# Display the summary of missing data
missing_summary

,Missing Values,Percentage
RespondentID,0,0.000000
Year,0,0.000000
Country,0,0.000000
Employment,1784,1.604187
UndergradMajor,12756,11.470295
DevType,10776,9.689863
LanguageWorkedWith,9191,8.264619
LanguageDesireNextYear,15165,13.636486
DatabaseWorkedWith,25350,22.794918
DatabaseDesireNextYear,36975,33.248208
